In [ ]:
import pandas as pd
from helpermodules import memory_handling as mh
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from functions import compute_sentiment, filtering_df, retrieve_datas, scraping_speeches, update_realtime, analysis
from functions.update_realtime import change_time
from functions.filtering_df import main as filtering
from functions.analysis import main as plot
from functions.scraping_speeches import main as scraping

#######
'''questa prima parte del codice presenta il codice che dovrebbe essere implementato 
(quello preceduto da '#') e in seguito il codice che ho utlizzato io non avendo ancora 
accesso alle funzioni, riciclando i file csv e 'pulendoli' per la task.
Le funzioni che verranno implementate devono restituire un oggetto che abbia 
la stessa forma e caratteristiche dell'oggetto finale (evidenziato nel codice)'''

yearlist = [2020,2021,2022,2023,2024]
# df_fed = scraping(yearlist) -> ALREADY RUN, STORE IN ANOTHER FILE
file = "2020-2024fedspeeches.pkl"
helper = mh.PickleHelper.pickle_load(file)
df_fed = helper.obj
df_fed['speaker'] = df_fed['speaker'].str.replace('Speech - ', '', regex=False)
df_fed['speaker'] = df_fed['speaker'].str.replace('Discussion - ', '', regex=False)

timezone1 = df_fed

#df_prices = retrieve_datas(df_speech, deltabefore, deltaafter) -> ALREADY RUN
file2 = "2020-2024prices.pkl"
helper = mh.PickleHelper.pickle_load(file2)
df = helper.obj
timezone2 = df
print(df.columns) 
df['date'] = df['datetime'].dt.date
df_prices = df


#df_speech = computespeech()
file3 = "2020-2024speeches.pkl"
helper = mh.PickleHelper.pickle_load(file3)
df_speech = helper.obj

df_speech = df_speech[df_speech['date']>='2020-01-01']
df_speech = df_speech.sort_values(['date','timestamp'], ascending=True) #<--- OGGETTO FINALE
timezone3 = df_speech


#df_sentiment = compute_sentiment()
file4 = "2020-2024sentiment.pkl"
helper = mh.PickleHelper.pickle_load(file4)
df_sentiment = helper.obj 


############

df_fed.rename(columns={'timestamp': 'opening_time'}, inplace=True)
#update the correct timestamp for df_speech
df_speech_final = change_time1(df_speech, df_fed)
'''#sorting the values
df_speech_final = df_speech_final.sort_values(['date','timestamp'], ascending=True)



# Define market open and close times (in ET)
market_open = pd.to_datetime('09:30:00', format='%H:%M:%S').time()
market_close = pd.to_datetime('16:00:00', format='%H:%M:%S').time()

# Extract time from 'timestamp'
df_speech_final['time'] = df_speech_final['timestamp'].dt.time

# Filter rows where 'time' is within market hours
df_filtered = df_speech_final[(df_speech_final['time'] >= market_open) & (df_speech_final['time'] < market_close)]
df_speech_final = df_filtered.drop(columns=['time'])



df_speech=df_speech_final'''

Index(['datetime', 'close', 'volume'], dtype='object')
The drop ratio is 16.15598885793872 %


"#sorting the values\ndf_speech_final = df_speech_final.sort_values(['date','timestamp'], ascending=True)\n\n\n\n# Define market open and close times (in ET)\nmarket_open = pd.to_datetime('09:30:00', format='%H:%M:%S').time()\nmarket_close = pd.to_datetime('16:00:00', format='%H:%M:%S').time()\n\n# Extract time from 'timestamp'\ndf_speech_final['time'] = df_speech_final['timestamp'].dt.time\n\n# Filter rows where 'time' is within market hours\ndf_filtered = df_speech_final[(df_speech_final['time'] >= market_open) & (df_speech_final['time'] < market_close)]\ndf_speech_final = df_filtered.drop(columns=['time'])\n\n\n\ndf_speech=df_speech_final"

In [15]:
def change_time1(df_speech, dfnew):
    """
    Adjusts the timestamps in the `df_speech` DataFrame based on the opening time
    from `dfnew` for each speech. Specifically, it updates the timestamp of the
    first event for each unique combination of speaker, date, and title to the
    opening time specified in `dfnew`, and shifts subsequent timestamps by one minute.

    Parameters:
    df_speech (pd.DataFrame): DataFrame containing speech details with columns like
                               'speaker', 'date', 'title', and 'timestamp'.
    dfnew (pd.DataFrame): DataFrame containing speech details with columns 'speaker',
                          'date', 'title', and 'opening_time' (the time when the speech begins).

    Returns:
    pd.DataFrame: A modified version of `df_speech` where the timestamps are updated
                  based on the opening time for matching speeches, and rows with no matches
                  are removed. The rows where the "timestamp" was updated will remain.

    Side Effects:
    - Adds a "check" column to track which rows have been updated.
    - Drops rows that didn't have a matching speech in `dfnew`.
    - Prints the drop ratio, which is the ratio of remaining unique texts after the update.
    """  

    # Ensuring the time columns share the same datetime format 
    df_speech['date'] = pd.to_datetime(df_speech['date'])
    dfnew['date'] = pd.to_datetime(dfnew['date'])

    # Add a "check" column to track updated rows
    df_speech['check'] = 0

    # Count the initial number of unique texts
    r2 = df_speech['text'].nunique()

    # Iterate over dfnew to update the timestamps in df_speech
    for i in range(len(dfnew)):
        # Extract details for the current speech
        speaker = dfnew.iloc[i]['speaker']
        date = dfnew.iloc[i]['date']
        title = dfnew.iloc[i]['title']
        newtime = pd.to_datetime(dfnew.iloc[i]['opening_time'])  # Ensure newtime is a Timestamp

        # Find rows in df_speech matching speaker, date, and title
        mask = (
        (df_speech['date'] == date) & ((df_speech['title']==title) | (df_speech['speaker']==speaker)))

        # Update the minimum timestamp and set the "check" flag
        if mask.any():
            min_idx = df_speech.loc[mask, 'timestamp'].idxmin()
            df_speech.at[min_idx, 'timestamp'] = newtime
            df_speech.loc[mask, 'timestamp'] = (
                newtime + pd.to_timedelta(range(len(df_speech[mask])), unit='min')
            )
            df_speech.loc[mask, 'check'] = 1

    # Drop rows that were not updated (check == 0)
    df_speech = df_speech[df_speech['check'] == 1].drop(columns=['check'])

    # Count the remaining unique texts and calculate the drop ratio
    r1 = df_speech['text'].nunique()
    print('The drop ratio is', (1-(r1 / r2)) * 100, '%')
    #update the timestamp values with the date value of the corresponding row 
    df_speech['timestamp'] = df_speech.apply(
    lambda row: row['date'].replace(hour=row['timestamp'].hour, 
                                    minute=row['timestamp'].minute, 
                                    second=row['timestamp'].second), 
    axis=1
)
    #updating the timezone info 
    df_speech['timestamp'] = df_speech['timestamp'].dt.tz_localize('America/New_York')

    df_speech.loc[df_speech['timestamp'].apply(lambda x: x.tzinfo.utcoffset(x) == pd.Timedelta(hours=-4)), 'timestamp'] += pd.Timedelta(hours=1)


    return df_speech


In [ ]:
df_speech_final